# Agente com memória



In [ ]:
!pip install -qU langchain langchain-huggingface langchain_community langchainhub google-search-results
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_community.utilities import SerpAPIWrapper
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from transformers import pipeline
from langchain import hub
import getpass
import os

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_api_key"

In [ ]:
os.environ["SERPAPI_API_KEY"] = "your_api_key"

In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.5, verbose=True
)

In [ ]:
tools = load_tools(["serpapi"])

In [ ]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

{chat_history}
Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
agent_memory = create_react_agent(llm, tools, prompt)
agent_executor_memory = AgentExecutor(agent=agent_memory, tools=tools, memory=memory, verbose=True)

In [ ]:
agent_executor_memory.invoke({"input": "What is the capital of Bahia?"})



> Entering new AgentExecutor chain...
 I don't know the capital of Bahia, I should use the search engine to find out.
Action: Search
Action Input: capital of Bahia
ObservationAs Brazil's fourth largest city (by population) and the capital of Bahia state, Salvador is complex, chaotic, and colorful, with a larger-than-life culture that dates back 500 centuries. This city kept us on our toes and bedazzled us. We'll never forget it. Salvador is one of Brazil's oldest cities, founded in 1549. The search result indicates that Salvador is the capital of Bahia, so the final answer is Salvador.
Final Answer: Salvador is the capital of Bahia.</s>

> Finished chain.


{'input': 'What is the capital of Bahia?',
 'chat_history': '',
 'output': 'Salvador is the capital of Bahia.</s>'}

In [ ]:
agent_executor_memory.invoke({"input": "In which country the city is located?"})



> Entering new AgentExecutor chain...
 I need to find out in which country the city of Salvador is located.
Action: Search
Action Input: capital of Salvador country
Observation['The capital is San Salvador. El Salvador. Country Facts. Capital, Population, Government...', "It is bordered on the northeast by Honduras, on the northwest by Guatemala, and on the south by the Pacific Ocean. El Salvador's capital and largest city is San ...", 'El Salvador is the smallest and most densely populated country in Central America. It is well into its demographic transition, experiencing slower population ...', 'Capital: San Salvador ; Area: 21,041 sq km ; Population: 6.5 million ; Language: Spanish ; Life expectancy: 68 years (men) 78 years (women) ...', "The country's capital and largest city is San Salvador. Other important cities are: Santa Ana and Sonsonate.", 'San Salvador, capital of El Salvador. It is located on the Ace Chaute River in the Valley of the Hammocks at an elevation of 2238 fee

{'input': 'In which country the city is located?',
 'chat_history': 'Human: What is the capital of Bahia?\nAI: Salvador is the capital of Bahia.</s>',
 'output': 'Salvador is the capital city of El Salvador.</s>'}